Fonte: Material disponibilizado pelo prof Gustavo Henrique Paetzold na 4ª Escola Avançada em Big Data Analysis

# **Instalando requerimentos**

In [1]:
!pip install transformers==3

     |████████████████████████████████| 754 kB 33.1 MB/s 
     |████████████████████████████████| 1.2 MB 47.8 MB/s 
     |████████████████████████████████| 3.0 MB 41.7 MB/s 
     |████████████████████████████████| 895 kB 32.0 MB/s 


# **Importações**

In [2]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import f1_score
from IPython.display import Image

# **Carregando o modelo e tokenizador a serem utilizados**

In [3]:
#Carregue o modelo e tokenizador a serem usados:
pre_trained_model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(pre_trained_model_name)
model = BertModel.from_pretrained(pre_trained_model_name)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

# **Criando a classe de armazenamento de dados**

In [4]:
#Classe de manipulação de dataset:
class Dataset(Dataset):

    #Construtor da classe:
    def __init__(self, sentences, labels, tokenizer, max_length):
        #Armazene as entradas que serão passadas ao modelo:
        self.sentences = sentences
        
        #Armazene as labels que serão utilizadas para treino/validação/teste:
        self.labels = labels
        
        #Armazene o tokenizador:
        self.tokenizer = tokenizer
        
        #Armazene o tamanho máximo das sentenças:
        self.max_length = max_length
    
    #Retorna o número de instâncias:
    def __len__(self):
        return len(self.sentences)
    
    #Retorna uma instância completa com base num índice:
    def __getitem__(self, index):
        #Obtenha a entrada do índice pertinente:
        sentence = self.sentences[index]
        
        #Obtenha a label do índice pertinente:
        label = self.labels[index]
        
        #Tokenize a entrada:
        encoding = self.tokenizer.encode_plus(
          sentence,
          add_special_tokens=True,
          max_length=self.max_length,
          return_token_type_ids=True,
          pad_to_max_length=True,
          return_attention_mask=True,
          return_tensors='pt',
          truncation=True
        )
    
        #Obtenha os códigos numéricos da sentença:
        input_ids = encoding['input_ids'].flatten()
        
        #Obtenha os códigos numéricos dos token types:
        token_type_ids = encoding['token_type_ids'].flatten()
        
        #Obtenha a máscara de atenção da sentença:
        attention_mask = encoding['attention_mask'].flatten()
        
        #Transforme a label da instância em um tensor:
        label_tensor = torch.tensor(label, dtype=torch.long)
        
        #Retorne um dicionário com estes dados:
        return {
          'input_ids': input_ids,
          'token_type_ids': token_type_ids,
          'attention_mask': attention_mask,
          'labels': label_tensor
        }

# **Criando o carregador de dados**

In [5]:
# Motando o drive para importar os datasets
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# Realiza as modificações no dataset importado
def GetProcessedDataset(df):
  #Convertendo 'summary' e 'description' para string
  df = df.astype({'summary':'str', 'description':'str'})

  #Filtrando severidades confiaveis
  df = df.loc[(df['severity_level'] != 'enhancement') & (df['severity_level'] != 'normal')]

  #Modificando severidade diferente de blocker (Classificação binaria)
  df.loc[(df['severity_level'] != 'blocker'), 'severity_level'] = "no-blocker"

  # Cria a coluna de label a partir de "severity_level" (do tipo int)
  df['label'] = df['severity_level'].rank(method='dense', ascending=False).astype(int)

  return df

#**Criando o modelo**

In [7]:
#Classificador de sentimento:
class SentimentClassifier(torch.nn.Module):

    #Construtor da classe
    def __init__(self, n_classes, pre_trained_model_name):
    
        #Inicialize o modelo:
        super(SentimentClassifier, self).__init__()
        
        #Carregue um modelo BERT pré-treinado:
        self.bert = BertModel.from_pretrained(pre_trained_model_name)
        
        #Crie a camada linear final para classificação:
        self.linear = torch.nn.Linear(self.bert.config.hidden_size, n_classes)

    #Função de execução do modelo:
    def forward(self, input_ids, attention_mask):
        #Passe a entrada pelo modelo BERT:
        word_outputs, pooled_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        
        #Passe a saída "pooled" do modelo BERT à camada linear:
        return self.linear(pooled_output)

# **Treinando o modelo**

In [8]:
def train_model(model, data_loader, loss_fn, optimizer, scheduler):
    #Coloque o modelo em modo de treinamento:
    model = model.train()
    
    #Inicialize o erro total da epoch:
    total_loss = 0
    total_preds = []
    
    #Para cada batch do data_loader, faça:
    for d in data_loader:
        #Obtenha os dados da batch:
        input_ids = d["input_ids"]
        attention_mask = d["attention_mask"]
        labels = d["labels"]
        
        #Passe os dados pelo modelo:
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        
        #Obtenha as predições:
        _, preds = torch.max(outputs, dim=1)
        total_preds.extend([p.item() for p in preds])
        
        #Calcule o erro:
        loss = loss_fn(outputs, labels)
        total_loss += loss.item()
        
        #Propague o erro para o modelo, promovendo aprendizado:
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
    return total_preds, total_loss

In [9]:
def test_model(model, data_loader, loss_fn):
    #Coloque o modelo em modo de treinamento:
    model = model.eval()
    
    #Inicialize o erro total da epoch:
    total_loss = 0
    total_preds = []
    
    #Para cada batch do data_loader, faça:
    for d in data_loader:
        #Obtenha os dados da batch:
        input_ids = d["input_ids"]
        attention_mask = d["attention_mask"]
        labels = d["labels"]
        
        #Passe os dados pelo modelo:
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        
        #Obtenha as predições:
        _, preds = torch.max(outputs, dim=1)
        total_preds.extend([p.item() for p in preds])
        
        #Calcule o erro:
        loss = loss_fn(outputs, labels)
        total_loss += loss.item()
    return total_preds, total_loss

**Treinando modelo nos Datasets**

In [10]:
# Temos 3 datasets para treinar
for datasetNum in range(3):
  print(f"===>Training dataset: {datasetNum}")

  # Carregando o dataset
  if datasetNum == 0:
    loadedDataset = pd.read_csv('/content/drive/My Drive/TrabFinal_Mineracao/Datasets/Eclipse_total.csv')
  elif datasetNum == 1:
    loadedDataset = pd.read_csv('/content/drive/My Drive/TrabFinal_Mineracao/Datasets/GCC_total.csv')
  elif datasetNum == 2:
    loadedDataset = pd.read_csv('/content/drive/My Drive/TrabFinal_Mineracao/Datasets/Mozilla_total.csv')

  # Realizando o processamento do dataset (pré-processamento)
  df = GetProcessedDataset(loadedDataset)

  #Carregando setences e labels
  sentences = df['description'].values
  labels = df['label'].values

  #Defina as proporções de treinamento/validação/teste:
  tr_prop = 0.9
  va_prop = 0.05
  te_prop = 0.05

  #Obtenha o tamanho do Dataset:
  dataset_size = len(sentences)

  #Calcules os índices de divisão de treinamento/validação/teste:
  tr_limit = int(tr_prop*dataset_size)
  va_limit = int((tr_prop+va_prop)*dataset_size)
  te_limit = int((tr_prop+va_prop+te_prop)*dataset_size)

  #Divida-os para treinamento/validação/teste:
  sentences_tr = sentences[:tr_limit]
  labels_tr = labels[:tr_limit]
  sentences_va = sentences[tr_limit:va_limit]
  labels_va = labels[tr_limit:va_limit]
  sentences_te = sentences[va_limit:te_limit]
  labels_te = labels[va_limit:te_limit]

  #Estabeleça parâmetros para o Dataset:
  max_length = 50
  batch_size = 32
  num_workers = 0 #Paramentro que o dataload recebe 

  #Crie Datasets para os dados:
  dtr = Dataset(sentences_tr, labels_tr, tokenizer, max_length)
  dva = Dataset(sentences_va, labels_va, tokenizer, max_length)
  dte = Dataset(sentences_te, labels_te, tokenizer, max_length)

  #Crie DataLoaders para os Datasets:
  dltr = DataLoader(dtr, batch_size=batch_size, num_workers=num_workers)
  dlva = DataLoader(dva, batch_size=batch_size, num_workers=num_workers)
  dlte = DataLoader(dte, batch_size=batch_size, num_workers=num_workers)

  #Crie o modelo:
  sentiment_classifier = SentimentClassifier(n_classes=7, pre_trained_model_name=pre_trained_model_name)

  #Crie elementos de treinamento:
  epochs = 1 #Numero de vezes que sera passado o dataset
  total_steps = len(dltr)*epochs
  loss_function = torch.nn.CrossEntropyLoss()
  optimizer = AdamW(sentiment_classifier.parameters())
  scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

  #Treinando:
  for i in range(epochs):
      #Treine em dados de treinamento:
      print(f"\tTreinando o modelo do dataset {datasetNum}, epoch: {i}")
      preds_tr, total_loss_tr = train_model(sentiment_classifier, dltr, loss_function, optimizer, scheduler)
      
      #Valide em dados de validação:
      print(f"\tValidando o modelo do dataset {datasetNum}, epoch: {i}")
      preds_va, total_loss_va = test_model(sentiment_classifier, dlva, loss_function)
      
      #Imprima os erros de treinamento/validação:
      print(f"\t\tErro de treinamento: {total_loss_tr}")
      print(f"\t\tErro de validação: {total_loss_va}")
    
  #Testando o modelo:
  preds_te, total_loss_te = test_model(sentiment_classifier, dlte, loss_function)

  #Calcule o erro com as etiquetas reais:
  f1 = f1_score(labels_te, preds_te, average='micro')
  print(f"\tDataset {datasetNum} F1-Score: {f1}")

===>Training dataset: 0
	Treinando o modelo do dataset 0, epoch: 0
	Validando o modelo do dataset 0, epoch: 0
		Erro de treinamento: 84.64152718894184
		Erro de validação: 3.8651434872299433
	Dataset 0 F1-Score: 1.0
===>Training dataset: 1
	Treinando o modelo do dataset 1, epoch: 0
	Validando o modelo do dataset 1, epoch: 0
		Erro de treinamento: 45.18206003587693
		Erro de validação: 2.743199400603771
	Dataset 1 F1-Score: 0.9782608695652174
===>Training dataset: 2
	Treinando o modelo do dataset 2, epoch: 0
	Validando o modelo do dataset 2, epoch: 0
		Erro de treinamento: 32.875183993950486
		Erro de validação: 1.1568964421749115
	Dataset 2 F1-Score: 0.9558823529411765
